# Reflection Pattern - Agentic AI Design Patterns 01

## I. Mô Hình Reflection (Phản Xạ)
### 1. Định nghĩa
- **Reflection** (Phản ánh) là một kỹ thuật trong trí tuệ nhân tạo (AI), đặc biệt áp dụng cho các mô hình ngôn ngữ lớn (Large Language Models - LLM), nhằm cho phép mô hình tự đánh giá và cải thiện đầu ra của chính mình. Đây là một quá trình lặp lại, mô phỏng cách con người suy nghĩ, học hỏi và tinh chỉnh công việc.

- Là quá trình bao gồm các bước sau:
	- LLM tạo ra phản hồi cho câu hỏi hoặc nhiệm vụ từ người dùng;
	- LLM tự phản ánh để đánh giá chất lượng, độ chính xác và tính đầy đủ của phản hồi ban đầu;
	- Dựa trên phản ánh đó, LLM tinh chỉnh phản hồi để trả về phiên bản tốt hơn;
	- Quá trình này có thể lặp lại nhiều lần để đạt được kết quả mong muốn.

<p align="center"><img src="./attachments/ReflectionPipe.png" alt="" width="700"/></p>

- Các thành phần:
	- Prompt: đầu vào của mô hình;
	- Generate: model tạo đầu ra dựa vào prompt;
	- Output: đầu ra của mô hình;
	- Reflect: phân tích, kiểm tra và bổ sung để tăng cường chất lượng đầu ra;
	- Reflected Text: đầu ra sau khi trải qua quá trình Reflect;
	- Iterate: qui trình Reflect được lặp đi lặp lại để tinh chỉnh kết quả cuối cùng;
	- Response: đầu ra cuối cùng sau khi trải qua quá trình Reflect.

### 2. Tầm Quan Trọng
- **Cải thiện chất lượng đầu ra**: Reflection giúp LLM tạo ra câu trả lời chính xác, chi tiết và phù hợp hơn.
- **Mô phỏng tư duy con người**: giống cách con người tự đánh giá và cải thiện, làm cho hành vi của AI trở nên trực quan hơn.
- **Tăng khả năng thích nghi**: cho phép AI điều chỉnh phản hồi dựa trên bối cảnh hoặc yêu cầu mới.

## II. Triển khai với LangChain
### 1. Các bước triển khai
- **LangChain** là một framework mạnh mẽ để phát triển ứng dụng sử dụng LLM.
	1. Khởi tạo mô hình: sử dụng GeminiAPI để minh họa;
	2. Định nghĩa prompt: bao gồm
		- `initial_prompt`: Để khởi tạo câu trả lời ban đầu của mô hình;
		- `reflection_promp`: Để thực hiện quá trình tinh chỉnh câu trả lời trước đó;
	3. Sử dụng RunnableSequence để liên kết quá trình thực hiện.
	4. Thực thi.

### 2. Minh họa

In [2]:
import os
import logging

from langchain import PromptTemplate
from langchain_core.runnables import RunnableSequence
from langchain_google_genai import ChatGoogleGenerativeAI
from IPython.display import Markdown, display

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [3]:
os.environ["GOOGLE_API_KEY"] = input("Your Gemini API key: ")

llm = ChatGoogleGenerativeAI(
    model       = "gemini-2.0-flash",
    temperature = 0.55,
    max_tokens  = None,
    timeout     = None,
    max_retries = 2,
)

**PROMPT**

In [4]:
#Initial Prompt
initial_prompt = PromptTemplate(
    input_variables = ["question"],
    template        = "Answer the following question: {question}"
)

In [27]:
#Reflection Prompt
reflection_prompt = PromptTemplate(
    input_variables=["initial_answer"],
    template="Review and improve the following answer to better address the question. Rewrite a detailed final answer, ignore unecessary information: {initial_answer}"
)

**PIPELINE**

In [28]:
#Initial Chain
initial_chain = RunnableSequence(initial_prompt | llm)

#Reflect Chain
reflection_chain = RunnableSequence(reflection_prompt | llm)

#PIPELINE
def run_reflection_pipeline(input_dict):
    try:
        #Step 1: Initial Response
        logger.info("Generating initial answer...")
        initial_answer = initial_chain.invoke({"question": input_dict["question"]})
        logger.info("Initial answer generated.")

        #Step 2: Apply reflection
        logger.info("Improving answer through reflection...")
        final_answer = reflection_chain.invoke({"initial_answer": initial_answer})
        logger.info("Final answer generated.")
        
        return {"text": final_answer}
    except Exception as e:
        logger.error("Error in reflection pipeline: %s", e)
        raise

In [29]:
question = "There are three boxes. One box contains only apples, one box contains only oranges, and one box contains both apples and oranges. All the boxes are labeled incorrectly. You are only allowed to take one fruit from any one box. Which box should you take a fruit from to determine the correct labels of all three boxes?"
result = run_reflection_pipeline({"question": question})

content = result["text"].content
display(Markdown(content))

INFO:__main__:Generating initial answer...
INFO:__main__:Initial answer generated.
INFO:__main__:Improving answer through reflection...
INFO:__main__:Final answer generated.


Here's a revised and more direct answer:

You must select a fruit from the box labeled "Apples and Oranges".  Since all the boxes are mislabeled, the box marked "Apples and Oranges" contains either only apples or only oranges.

*   **If you pick an apple:**  The box you selected *must* contain only apples.  Because the box labeled "Oranges" is also mislabeled, it cannot contain only oranges. Therefore, the box labeled "Oranges" must contain the mixed fruit (apples and oranges). This leaves the box labeled "Apples" to contain only oranges.

*   **If you pick an orange:** The box you selected *must* contain only oranges. Because the box labeled "Apples" is mislabeled, it cannot contain only apples. Therefore, the box labeled "Apples" must contain the mixed fruit (apples and oranges). This leaves the box labeled "Oranges" to contain only apples.

By picking a single fruit from the box labeled "Apples and Oranges", you immediately determine its true contents, allowing you to deduce the contents of the remaining two boxes.

****